In [15]:
import torch as T
import torch.nn as nn 
import os
project_path= os.path.dirname(os.path.abspath(os.curdir))
import sys
sys.path.insert(0, project_path+ '/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork
project_path

'/Users/niyi/Documents/GitHub/Optimal-Control'

In [16]:
test_state= T.rand((5, 4))
test_action= T.rand((5, 2))

In [17]:
critic_save_path= os.path.join(project_path, 'Data', 'Temp', 'critic_data')
actor_save_path= os.path.join(project_path, 'Data', 'Temp', 'actor_data')
observation_size= 4
action_size= 2
critic_layers= (10, 15)
critic_activations= (nn.ReLU(), nn.ReLU())
critic_learning_rate= 0.02
test_critic1 = CriticNetwork(observation_size,
                             action_size,
                             critic_layers,
                             critic_activations,
                             'test_critic',
                             critic_learning_rate,
                             critic_save_path)
test_critic1.save_checkpoint()
print(test_critic1(test_state, test_action))
test_critic1.load_checkpoint()
print(test_critic1(test_state, test_action))
test_critic1.checkpoint_file

... saving checkpoint ...
tensor([[0.7581],
        [0.7217],
        [0.6977],
        [0.9210],
        [0.8352]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[0.7581],
        [0.7217],
        [0.6977],
        [0.9210],
        [0.8352]], grad_fn=<AddmmBackward0>)


'/Users/niyi/Documents/GitHub/Optimal-Control/Data/Temp/critic_data/test_critic'

In [18]:
actor_layers= (10, 15)
actor_activations= (nn.ReLU(), nn.ReLU())
actor_learning_rate= 0.02
test_actor1 = ActorNetwork(observation_size,
                             action_size,
                             actor_layers,
                             actor_activations,
                             'test_critic',
                             actor_learning_rate,
                             actor_save_path)
test_actor1.save_checkpoint()
print(test_actor1(test_state))
test_actor1.load_checkpoint()
print(test_actor1(test_state))
test_actor1.checkpoint_file

... saving checkpoint ...
tensor([[-0.3029,  0.1755],
        [-0.4778,  0.2025],
        [-0.4362,  0.1010],
        [-0.4249,  0.1304],
        [-0.5176,  0.1580]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[-0.3029,  0.1755],
        [-0.4778,  0.2025],
        [-0.4362,  0.1010],
        [-0.4249,  0.1304],
        [-0.5176,  0.1580]], grad_fn=<AddmmBackward0>)


'/Users/niyi/Documents/GitHub/Optimal-Control/Data/Temp/actor_data/test_critic'

**3. Functional tests** to check if this module functions as intended.They only verify the output of an action and do not check the intermediate states of the system when performing that action. 

Can copy parameters

In [19]:
test_critic2= CriticNetwork(observation_size,
                             action_size,
                             critic_layers,
                             critic_activations,
                             'test_critic2',
                             critic_learning_rate,
                             critic_save_path)

print(test_critic2(test_state, test_action))
for target_param, param in zip(test_critic2.parameters(), test_critic1.parameters()):
    target_param.data.copy_(param.data)
print(test_critic2(test_state, test_action))

tensor([[-0.0974],
        [-0.0271],
        [-0.1013],
        [ 0.0128],
        [-0.0364]], grad_fn=<AddmmBackward0>)
tensor([[0.7581],
        [0.7217],
        [0.6977],
        [0.9210],
        [0.8352]], grad_fn=<AddmmBackward0>)


In [20]:
test_actor2= ActorNetwork(observation_size,
                             action_size,
                             actor_layers,
                             actor_activations,
                             'test_critic',
                             actor_learning_rate,
                             actor_save_path)
print(test_actor2(test_state))
for target_param, param in zip(test_actor2.parameters(), test_actor1.parameters()):
    target_param.data.copy_(param.data)
print(test_actor2(test_state))

tensor([[-0.0743,  0.3329],
        [ 0.1989,  0.3361],
        [-0.0132,  0.2911],
        [ 0.0090,  0.4133],
        [ 0.2157,  0.3326]], grad_fn=<AddmmBackward0>)
tensor([[-0.3029,  0.1755],
        [-0.4778,  0.2025],
        [-0.4362,  0.1010],
        [-0.4249,  0.1304],
        [-0.5176,  0.1580]], grad_fn=<AddmmBackward0>)


Can learn to approximate values

In [21]:
y_train= T.rand((5, 1))
epochs = 300
for epoch in range(epochs):
    y_pred = test_critic1(test_state, test_action)
    loss = T.nn.MSELoss()(y_pred, y_train)
    test_critic1.optimizer.zero_grad()
    loss.backward()
    test_critic1.optimizer.step()

test_critic1(test_state, test_action), y_train

(tensor([[0.6643],
         [0.0303],
         [0.4405],
         [0.3337],
         [0.2818]], grad_fn=<AddmmBackward0>),
 tensor([[0.6643],
         [0.0303],
         [0.4405],
         [0.3337],
         [0.2818]]))

In [22]:
epochs = 300
for epoch in range(epochs):
    y_pred = test_actor1(test_state)
    loss = T.nn.MSELoss()(y_pred, test_action)
    test_actor1.optimizer.zero_grad()
    loss.backward()
    test_actor1.optimizer.step()

test_actor1(test_state), test_action

(tensor([[0.8183, 0.9644],
         [0.4772, 0.0145],
         [0.4895, 0.2994],
         [0.1575, 0.2199],
         [0.1056, 0.5755]], grad_fn=<AddmmBackward0>),
 tensor([[0.8183, 0.9644],
         [0.4772, 0.0145],
         [0.4895, 0.2994],
         [0.1575, 0.2199],
         [0.1056, 0.5755]]))